In [ ]:
from dotenv import load_dotenv,find_dotenv
import os
load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
if not GEMINI_API_KEY:
   raise ValueError('GEMINI_API_KEY is not set in the env file')
print('GEMINI_API_KEY loaded successfully')    


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from IPython.display import display,Markdown
llm = ChatGoogleGenerativeAI(api_key=GEMINI_API_KEY,temperature=0,model='gemini-1.5-flash')
resp = llm.invoke('what is the meaning of life?') 
display(Markdown(resp.content))

In [ ]:
from typing_extensions import TypedDict
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import StateGraph,START,END
import json
from IPython.display import display,Image
from gtts import gTTS
import os
from datetime import datetime
import logging
import requests
from langchain_google_genai import ChatGoogleGenerativeAI
from IPython.display import display,Markdown
from dotenv import load_dotenv,find_dotenv
import os
load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
if not GEMINI_API_KEY:
   raise ValueError('GEMINI_API_KEY is not set in the env file')
print('GEMINI_API_KEY loaded successfully')    

llm = ChatGoogleGenerativeAI(api_key=GEMINI_API_KEY,temperature=0,model='gemini-1.5-flash')

class GraphState(TypedDict):
    story_theme: str
    generated_story: str
    character_list:str
    plot:list[str]
    image_prompts: list[str] 
    voice_saved_path:str    
    image_saved_path:str
    
def generate_story(state:GraphState)->GraphState:
    story_theme = state['story_theme']
    generate_story_prompt ="""
    Imagine you are a skilled storyteller tasked with creating a simple and interesting story based on the theme below.  
  
    Theme: {theme}
    Your goal is to generate a detailed and realistic story
    The story should have:
    - A clear beginning, middle, and end.
    - Easy-to-understand language.
    - Detail and meaningful sentences.
    - Interesting characters and events.
    
    Story:
    """
    sys_msg = SystemMessage(content="You are a storyteller who writes clear and engaging stories in simple English.")
    hum_msg =  HumanMessage(content=generate_story_prompt.format(theme=story_theme))
    print('Generating story for theme')
    resp = llm.invoke([sys_msg,hum_msg])
    state['generated_story'] = resp.content
    return state 
def generate_story_char(state:GraphState)->GraphState:
    story = state['generated_story'] 
    generate_description_prompt = '''Based on the Story {story}, create a brief description of the main and supporting character, object, or scene. Include specific details about appearance, characteristics, and any unique features. This description will be used to maintain consistency across multiple images.'''
    sys_msg = SystemMessage(content="You are an assistant that extracts all character names and details from a story.")
    hum_msg =  HumanMessage(content=generate_description_prompt.format(story=story))
    print('Generating story Characters')
    resp = llm.invoke([sys_msg,hum_msg])
    state['character_list'] = resp.content
    return state

def generate_plot(state: GraphState) -> GraphState:
    story = state['generated_story']
    character_description =  state['character_list']
    generate_description_prompt = f"""create a short, 5-step plot for a video based on this story: '{story}' and featuring this description: {character_description}. Each step should be a brief description of a single frame, maintaining consistency throughout. Keep it family-friendly and avoid any sensitive themes.
      ....

    Return the plot as a in JSON, formatted as follows:
    ["Step 1 description", "Step 2 description", "Step 3 description", "Step 4 description", "Step 5 description"]
    """
    hum_msg = HumanMessage(content=generate_description_prompt.format(story=story,character_description=character_description))
    print('Generating scene description of the story')
    resp = llm.invoke([hum_msg]) 
    plot_string = resp.content
    json_string = plot_string.strip().replace('```json', '').replace('```', '').strip()
    plot_list = json.loads(json_string)
    state['plot']=plot_list
    return state 
def generate_image_prompts(state: GraphState) -> GraphState:
    plot =  state['plot']
    character_description =  state['character_list']
    generate_description_prompt = f"""Based on this plot: '{plot}' and featuring this description: {character_description}, generate 5 specific, family-friendly image prompts, one for each step. Each prompt should be detailed enough for image generation, maintaining consistency, and suitable for DALL-E. 

                                    Always include the following in EVERY prompt to maintain consistency:
                                    1. A brief reminder of the main character or object's key features
                                    2. The specific action or scene described in the plot step
                                    3. Any relevant background or environmental details

                                    Format each prompt as a numbered list item, like this:
                                    1. [Your prompt here]
                                    2. [Your prompt here]
                                    ... and so on.
                                        """
    

    hum_msg = HumanMessage(content=generate_description_prompt.format(character_description=character_description, plot=plot))
    print('Generating image prompts for the story')
    response = llm.invoke([hum_msg])
    prompts = []
    for line in response.content.split("\n"):
        if line.strip().startswith(("1.", "2.", "3.", "4.", "5.")):
            prompt = line.split(".", 1)[1].strip()
            prompts.append(
                f"Create a detailed, photorealistic image of the following scene: {prompt}"
            )

    if len(prompts) != 5:
        raise ValueError(
            f"Expected 5 prompts, but got {len(prompts)}. Please try again."
        )

    state["image_prompts"] = prompts
    return state

def voice_generates(text: str, file_name: str, folder: str):
    try:
        tts = gTTS(text)
        if not os.path.exists(folder):
            os.makedirs(folder)
        save_path = os.path.join(folder, file_name)
        tts.save(save_path)
        return save_path
    except Exception as e:
        logging.error(f"Unexpected error: {e}")
        return None


def finalvoicefun(sceneList):
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
    dynamic_folder_voice = f"voice_{timestamp}"
    output_folder_voice = os.path.join("Generated_voice", dynamic_folder_voice)
    voice_paths = []
    for i, scene in enumerate(sceneList):
        try:
            voice_path = voice_generates(
                scene, f"voice_scene_{i+1}.mp3", output_folder_voice
            )

            if voice_path:
                voice_paths.append(voice_path)
                print(f"Voiceover saved locally: {voice_path}")
            else:
                print(f"Failed to save voiceover for scene {i+1}.")
        except Exception as e:
            print(f"Error generating voice for scene {i+1}: {e}")
    return output_folder_voice    

def generate_scence_voice(state:GraphState)->GraphState:
    scene_list = state['plot']
    resp = finalvoicefun(scene_list) 
    print(f"Generating voice{resp}")
    return {'voice_saved_path':resp}
    # state['voice_saved_path'] = resp
    # return state

def saveImage(image_content, save_path):
    output_folder = os.path.dirname(save_path)
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    with open(save_path, "wb") as f:
        f.write(image_content)


def images_generates(prompt: str):
    try:
        # response = client.images.generate(
        #     model=imagemodel,
        #     prompt=prompt,
        #     quality="standard",
        #     n=1,
        # )
        # image_url = response.data[0].url
        image_url = "https://www.w3schools.com/w3images/lights.jpg"
        return image_url

    except Exception as e:
        logging.error(f"Unexpected error generating image: {e}")
        raise


def finalscenesfun(image_prompts):
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
    dynamic_folder = f"images_{timestamp}"
    output_folder = os.path.join("Generated_images", dynamic_folder)

    for i, scene in enumerate(image_prompts):
        try:
             
            imageUrl = images_generates(scene)
            if not imageUrl:
                raise ValueError("Generated image URL is empty.")

            imageContent = requests.get(imageUrl).content
            if not imageContent:
                raise ValueError("Failed to retrieve image content.")

            save_path = os.path.join(output_folder, f"image_scene_{i+1}.png")
            saveImage(imageContent, save_path)
            print(f"Images saved to {save_path}")

        except Exception as e:
            logging.error(f"Error processing scene {i+1}: {e}")
            continue

    return output_folder

def generate_scence_image(state:GraphState)->GraphState:
    image_prompts = state['image_prompts']
    resp = finalscenesfun(image_prompts) 
    state["image_saved_path"]=resp
    print(f"Generating images path: {resp}")
    return {"image_saved_path": resp}
    # state['image_saved_path'] = resp
    # return state

def final_video(state:GraphState)->GraphState:
   
    return state

workflow = StateGraph(GraphState)
workflow.add_node('generate_story',generate_story)
workflow.add_node('generate_story_char',generate_story_char)
workflow.add_node('generate_plot',generate_plot)
workflow.add_node('generate_image_prompts',generate_image_prompts)
workflow.add_node('generate_scence_voice',generate_scence_voice)
workflow.add_node('generate_scence_image',generate_scence_image)
workflow.add_node('final_video',final_video)

workflow.add_edge(START,'generate_story')
workflow.add_edge('generate_story','generate_story_char')
workflow.add_edge('generate_story_char','generate_plot')
workflow.add_edge('generate_plot','generate_image_prompts')
workflow.add_edge('generate_plot','generate_scence_voice')
workflow.add_edge('generate_image_prompts','generate_scence_image')
 
workflow.add_edge(['generate_scence_voice','generate_scence_image'],'final_video') 
workflow.add_edge('final_video',END) 

app = workflow.compile()
# display(Image(app.get_graph().draw_mermaid_png()))
story_theme =  """A 24 year old boy seeing out from the train’s window shouted…

“Dad, look the trees are going behind!”

Dad smiled and a young couple sitting nearby, looked at the 24 year old’s childish behavior with pity, suddenly he again exclaimed…

“Dad, look the clouds are running with us!”

The couple couldn’t resist and said to the old man…

“Why don’t you take your son to a good doctor?” The old man smiled and said…“I did and we are just coming from the hospital, my son was blind from birth, he just got his eyes today.”

Every single person on the planet has a story. Don’t judge people before you truly know them. The truth might surprise you."""
resp = app.invoke({'story_theme':story_theme})
print(resp)
 

In [ ]:
print(resp['image_prompts'][4
                    ])  
print('hi')  
   
     

In [ ]:
import requests, base64
from dotenv import load_dotenv,find_dotenv
import os
load_dotenv()
NVIDIA_API_KEY = os.getenv("NVIDIA_API_KEYs")
if not NVIDIA_API_KEY:
   raise ValueError('NVIDIA_API_KEY is not set in the env file')
print('NVIDIA_API_KEY loaded successfully')   
invoke_url = "https://ai.api.nvidia.com/v1/genai/nvidia/consistory"
headers = {
    "Authorization": f"Bearer {NVIDIA_API_KEY}",
    "Accept": "application/json",
}

payload = {
    "mode": 'init',
    "subject_prompt": "A 24-year-old man with wide, curious eyes, experiencing sight for the first time.",
    "subject_tokens":["man", "curly hair", "glasses", "blue jacket"],
    "subject_seed": 24,
    "style_prompt": "A highly detailed photorealistic portrait",
    "scene_prompt1": "Leo pressing his face against a cool train window, excitedly pointing at the passing landscape.",
    "negative_prompt": "blurry, distorted, unrealistic",
    "cfg_scale": 5,
    "same_initial_noise": False,
    "image_width": 1080,  # Set width for Shorts
    "image_height": 1920,  # Set height for Shorts
}

response = requests.post(invoke_url, headers=headers, json=payload)
response.raise_for_status()

data = response.json()

for idx, img_data in enumerate(data['artifacts']):
  img_base64 = img_data["base64"]
  img_bytes = base64.b64decode(img_base64)
  with open(f'{idx}.jpg', "wb") as f:
      f.write(img_bytes)

In [ ]:
import requests, base64
from dotenv import load_dotenv,find_dotenv
import os
load_dotenv()
NVIDIA_API_KEY = os.getenv("NVIDIA_API_KEY")
if not NVIDIA_API_KEY:
   raise ValueError('NVIDIA_API_KEY is not set in the env file')
print('NVIDIA_API_KEY loaded successfully')   
invoke_url = "https://ai.api.nvidia.com/v1/genai/nvidia/consistory"
headers = {
    "Authorization": f"Bearer {NVIDIA_API_KEY}",
    "Accept": "application/json",
}

 
payload = {
    "mode": 'init',
    "subject_prompt": "A 24-year-old man with wide, curious eyes, experiencing sight for the first time.",
    "subject_tokens":["man", "curly hair", "glasses", "blue jacket"],
    "subject_seed": 43,
    "style_prompt": "A highly detailed photorealistic portrait",
    "scene_prompt1": "Leo pressing his face against a cool train window, excitedly pointing at the passing landscape.",
    "negative_prompt": "blurry, distorted, unrealistic",
    "cfg_scale": 5,
    "same_initial_noise": False,
 
}

response = requests.post(invoke_url, headers=headers, json=payload)
response.raise_for_status()

data = response.json()

for idx, img_data in enumerate(data['artifacts']):
  img_base64 = img_data["base64"]
  img_bytes = base64.b64decode(img_base64)
  with open(f'{idx}.jpg', "wb") as f:
      f.write(img_bytes)


In [ ]:
from typing_extensions import TypedDict
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import StateGraph,START,END

class GraphState(TypedDict):
    story_theme: str
    generated_story: str
    
def generate_story(state:GraphState)->GraphState:
    story_theme = state['story_theme']
    generate_story_prompt ="""
    Imagine you are a skilled storyteller tasked with creating a simple and interesting story based on the theme below.  
  
    Theme: {theme}
    Your goal is to generate a detailed and realistic story
    The story should have:
    - A clear beginning, middle, and end.
    - Easy-to-understand language.
    - Detail and meaningful sentences.
    - Interesting characters and events.
    
    Story:
    """
    sys_msg = SystemMessage(content="You are a storyteller who writes clear and engaging stories in simple English.")
    hum_msg =  HumanMessage(content=generate_story_prompt.format(theme=story_theme))
    resp = llm.invoke([sys_msg,hum_msg])
    state['generated_story'] = resp.content
    return state 


workflow = StateGraph(GraphState)
workflow.add_node('generate_story',generate_story)
workflow.add_edge(START,'generate_story')
workflow.add_edge('generate_story',END)
app = workflow.compile()

story_theme =  "Once upon a time, there was a boy who lived in a jungle. One day, he saw a lion cub that was very hungry. The boy gave food to the lion cub, and they became friends."
resp = app.invoke({'story_theme':story_theme})
print(resp)

In [ ]:
from typing_extensions import TypedDict
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import StateGraph,START,END

class GraphState(TypedDict):
    story: str
    charlist: list
    
def generate_story_char(state:GraphState)->GraphState:
    story = state['story'] 
    generate_description_prompt = '''Based on the Story {story}, create a brief description of the main and supporting character, object, or scene. Include specific details about appearance, characteristics, and any unique features. This description will be used to maintain consistency across multiple images.'''
 
    sys_msg = SystemMessage(content="You are an assistant that extracts all character names and details from a story.")
    hum_msg =  HumanMessage(content=generate_description_prompt.format(story=story))
    resp = llm.invoke([sys_msg,hum_msg])
    state['charlist'] = resp.content
    return state 


workflow = StateGraph(GraphState)
workflow.add_node('generate_story_char',generate_story_char)
workflow.add_edge(START,'generate_story_char')
workflow.add_edge('generate_story_char',END)
app = workflow.compile()

story =  """Once upon a time, deep in a lush green jungle, lived a boy named Mika. Mika wasn't like the other children in his village. While they played near their homes, Mika loved exploring the jungle, its secrets whispering to him in the rustling leaves and the calls of unseen birds. He knew the paths like the back of his hand, and the jungle felt like a second home.

One sweltering afternoon, while searching for juicy mangoes, Mika stumbled upon a tiny lion cub, huddled under a giant banyan tree. The cub was thin, its ribs showing through its tawny fur. Its eyes, usually bright and playful, were dull with hunger. A whimper escaped its tiny mouth. Mika’s heart ached. He knew what hunger felt like; sometimes his family didn't have enough to eat.

Carefully, Mika approached the cub. He offered it a half-eaten mango, holding it out gently in his palm. The cub, hesitant at first, sniffed the fruit cautiously before lapping at the sweet juice. Mika smiled. He shared the rest of his mangoes with the cub, and then, remembering the sweet potatoes his mother had packed for him, he offered those too. The cub ate greedily, its tail thumping weakly against the ground.

Day after day, Mika visited the cub, bringing it food he found or shared from his own meals. He named the cub Leo. Slowly, Leo grew stronger, his fur regaining its shine, his eyes sparkling with life. They played together, Mika chasing Leo through the undergrowth, Leo playfully batting at Mika's legs. A unique friendship blossomed between a boy and a lion cub, a bond forged in shared hunger and kindness.

One day, Mika’s father saw him playing with Leo. He was worried at first, but Mika explained their friendship. To his surprise, his father understood. He knew the jungle was full of wonders, and that sometimes, the greatest friendships came from the most unexpected places. Mika and Leo continued their adventures, their bond a testament to the power of kindness and the magic of the jungle. And so, the boy who lived in the jungle and his lion cub friend lived happily, their story whispered on the jungle breeze."""
resp = app.invoke({'story':story})
print(resp['charlist'])
print(type(resp['charlist']))

In [ ]:
from typing_extensions import TypedDict
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import StateGraph, START, END
import ast
from langchain_core.pydantic_v1 import BaseModel, Field
import json
class GraphState(TypedDict):
    story: str
    plot: list[str]   



 

def generate_plot(state: GraphState) -> GraphState:
    story = state['story']
    character_description = """**Main Characters:**

    * **Mika:** A boy who lives in a lush green jungle. He is adventurous, kind, and resourceful, possessing intimate knowledge of the jungle's secrets.  He is depicted as having a compassionate heart, evident in his care for Leo.  No specific physical description is given.

    * **Leo:** A tiny lion cub, initially weak and undernourished, with thin, dusty fur and dull eyes. As he recovers, his fur becomes thicker and shinier, and his eyes sparkle with playful energy.

    * **Leo's Mother:** A large lioness with a magnificent mane.  She is depicted as cautious but ultimately grateful and understanding of Mika's kindness.


    **Supporting Characters/Elements:**

    * **Monkeys:**  Inhabit the jungle trees and are Mika's friends. No specific details are given about their appearance.

    * **Rhinoceroses:** Described as "grumpy" and inhabiting the jungle. No specific details are given about their appearance.

    * **Banyan Tree:** A large tree where Mika finds the lion cub.

    **Objects:**

    * **Mangoes:** Juicy mangoes that Mika shares with Leo.

    * **Berries:**  Food collected by Mika and shared with Leo.
    ...
    * **Banyan Tree Scene:** The scene where Mika discovers the weak lion cub under the banyan tree.

    * **Final Scene:** Mika observes Leo and his mother together, thriving in the jungle."""
    generate_description_prompt = f"""create a short, 5-step plot for a video based on this story: '{story}' and featuring this description: {character_description}. Each step should be a brief description of a single frame, maintaining consistency throughout. Keep it family-friendly and avoid any sensitive themes.
      ....

    Return the plot as a in JSON, formatted as follows:
    ["Step 1 description", "Step 2 description", "Step 3 description", "Step 4 description", "Step 5 description"]
    """
    

    hum_msg = HumanMessage(content=generate_description_prompt.format(story=story,character_description=character_description))
    resp = llm.invoke([hum_msg])

    # Parse the LLM's response into a list of strings
    plot_string = resp.content
    json_string = plot_string.strip().replace('```json', '').replace('```', '').strip()
    plot_list = json.loads(json_string)
    state['plot']=plot_list
    return state 


workflow = StateGraph(GraphState)
workflow.add_node('generate_plot', generate_plot)
workflow.add_edge(START, 'generate_plot')
workflow.add_edge('generate_plot', END)
app = workflow.compile()

story = """Once upon a time, deep in a lush green jungle, lived a boy named Mika. Mika wasn't like the other children in his village. While they played near their homes, Mika loved exploring the jungle, its secrets whispering to him in the rustling leaves and the calls of unseen birds. He knew the paths like the back of his hand, and the jungle felt like a second home.

One sweltering afternoon, while searching for juicy mangoes, Mika stumbled upon a tiny lion cub, huddled under a giant banyan tree. The cub was thin, its ribs showing through its tawny fur. Its eyes, usually bright and playful, were dull with hunger. A whimper escaped its tiny mouth. Mika’s heart ached. He knew what hunger felt like; sometimes his family didn't have enough to eat.

Carefully, Mika approached the cub. He offered it a half-eaten mango, holding it out gently in his palm. The cub, hesitant at first, sniffed the fruit cautiously before lapping at the sweet juice. Mika smiled. He shared the rest of his mangoes with the cub, and then, remembering the sweet potatoes his mother had packed for him, he offered those too. The cub ate greedily, its tail thumping weakly against the ground.

Day after day, Mika visited the cub, bringing it food he found or shared from his own meals. He named the cub Leo. Slowly, Leo grew stronger, his fur regaining its shine, his eyes sparkling with life. They played together, Mika chasing Leo through the undergrowth, Leo playfully batting at Mika's legs. A unique friendship blossomed between a boy and a lion cub, a bond forged in shared hunger and kindness.

One day, Mika’s father saw him playing with Leo. He was worried at first, but Mika explained their friendship. To his surprise, his father understood. He knew the jungle was full of wonders, and that sometimes, the greatest friendships came from the most unexpected places. Mika and Leo continued their adventures, their bond a testament to the power of kindness and the magic of the jungle. And so, the boy who lived in the jungle and his lion cub friend lived happily, their story whispered on the jungle breeze."""

resp = app.invoke({'story': story})
print(resp['plot'])
print(type(resp['plot']))

In [ ]:
from typing_extensions import TypedDict
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import StateGraph, START, END
import ast
from langchain_core.pydantic_v1 import BaseModel, Field
import json
class GraphState(TypedDict):
    story: str
    plot: list[str]  
    image_prompts: list[str] 



 

def generate_plot(state: GraphState) -> GraphState:
    plot = '''['Mika, a boy with kind eyes, explores the lush jungle, playfully interacting with a troop of monkeys swinging through the trees.  A majestic banyan tree looms in the background.', "Mika discovers a weak, thin lion cub, Leo, whimpering under the banyan tree.  Leo's dull eyes and ribs showing through his fur highlight his hunger.", 'Mika gently shares his mangoes and later sweet potatoes with the hungry cub. Leo cautiously eats, his tail thumping weakly as he gains strength.', "Days later, a healthy Leo playfully wrestles with Mika amidst the jungle undergrowth.  Mika's father watches from afar, initially concerned but then understanding.", 'Mika watches from a distance as Leo, now strong and vibrant, plays with his mother, a magnificent lioness, under the banyan tree.  The scene is peaceful and joyful.']'''
    character_description = """**Main Characters:**

    * **Mika:** A boy who lives in a lush green jungle. He is adventurous, kind, and resourceful, possessing intimate knowledge of the jungle's secrets.  He is depicted as having a compassionate heart, evident in his care for Leo.  No specific physical description is given.

    * **Leo:** A tiny lion cub, initially weak and undernourished, with thin, dusty fur and dull eyes. As he recovers, his fur becomes thicker and shinier, and his eyes sparkle with playful energy.

    * **Leo's Mother:** A large lioness with a magnificent mane.  She is depicted as cautious but ultimately grateful and understanding of Mika's kindness.


    **Supporting Characters/Elements:**

    * **Monkeys:**  Inhabit the jungle trees and are Mika's friends. No specific details are given about their appearance.

    * **Rhinoceroses:** Described as "grumpy" and inhabiting the jungle. No specific details are given about their appearance.

    * **Banyan Tree:** A large tree where Mika finds the lion cub.

    **Objects:**

    * **Mangoes:** Juicy mangoes that Mika shares with Leo.

    * **Berries:**  Food collected by Mika and shared with Leo.
    ...
    * **Banyan Tree Scene:** The scene where Mika discovers the weak lion cub under the banyan tree.

    * **Final Scene:** Mika observes Leo and his mother together, thriving in the jungle."""
    generate_description_prompt = f"""Based on this plot: '{plot}' and featuring this description: {character_description}, generate 5 specific, family-friendly image prompts, one for each step. Each prompt should be detailed enough for image generation, maintaining consistency, and suitable for DALL-E. 

                                    Always include the following in EVERY prompt to maintain consistency:
                                    1. A brief reminder of the main character or object's key features
                                    2. The specific action or scene described in the plot step
                                    3. Any relevant background or environmental details

                                    Format each prompt as a numbered list item, like this:
                                    1. [Your prompt here]
                                    2. [Your prompt here]
                                    ... and so on.
                                        """
    

    hum_msg = HumanMessage(content=generate_description_prompt.format(character_description=character_description, plot=plot))
    response = llm.invoke([hum_msg])
    prompts = []
    for line in response.content.split("\n"):
        if line.strip().startswith(("1.", "2.", "3.", "4.", "5.")):
            prompt = line.split(".", 1)[1].strip()
            prompts.append(
                f"Create a detailed, photorealistic image of the following scene: {prompt}"
            )

    if len(prompts) != 5:
        raise ValueError(
            f"Expected 5 prompts, but got {len(prompts)}. Please try again."
        )

    state["image_prompts"] = prompts
    return state
  

workflow = StateGraph(GraphState)
workflow.add_node('generate_plot', generate_plot)
workflow.add_edge(START, 'generate_plot')
workflow.add_edge('generate_plot', END)
app = workflow.compile()

story = """Once upon a time, deep in a lush green jungle, lived a boy named Mika. Mika wasn't like the other children in his village. While they played near their homes, Mika loved exploring the jungle, its secrets whispering to him in the rustling leaves and the calls of unseen birds. He knew the paths like the back of his hand, and the jungle felt like a second home.

One sweltering afternoon, while searching for juicy mangoes, Mika stumbled upon a tiny lion cub, huddled under a giant banyan tree. The cub was thin, its ribs showing through its tawny fur. Its eyes, usually bright and playful, were dull with hunger. A whimper escaped its tiny mouth. Mika’s heart ached. He knew what hunger felt like; sometimes his family didn't have enough to eat.

Carefully, Mika approached the cub. He offered it a half-eaten mango, holding it out gently in his palm. The cub, hesitant at first, sniffed the fruit cautiously before lapping at the sweet juice. Mika smiled. He shared the rest of his mangoes with the cub, and then, remembering the sweet potatoes his mother had packed for him, he offered those too. The cub ate greedily, its tail thumping weakly against the ground.

Day after day, Mika visited the cub, bringing it food he found or shared from his own meals. He named the cub Leo. Slowly, Leo grew stronger, his fur regaining its shine, his eyes sparkling with life. They played together, Mika chasing Leo through the undergrowth, Leo playfully batting at Mika's legs. A unique friendship blossomed between a boy and a lion cub, a bond forged in shared hunger and kindness.

One day, Mika’s father saw him playing with Leo. He was worried at first, but Mika explained their friendship. To his surprise, his father understood. He knew the jungle was full of wonders, and that sometimes, the greatest friendships came from the most unexpected places. Mika and Leo continued their adventures, their bond a testament to the power of kindness and the magic of the jungle. And so, the boy who lived in the jungle and his lion cub friend lived happily, their story whispered on the jungle breeze."""

resp = app.invoke({'story': story})
print(resp['image_prompts'])
print(type(resp['image_prompts']))

In [ ]:
from typing_extensions import TypedDict
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import StateGraph,START,END
from datetime import datetime
class GraphState(TypedDict): 
    sceneList: list[str]


def voice_generates(text: str, file_name: str, folder: str):
    try:
        tts = gTTS(text)
        if not os.path.exists(folder):
            os.makedirs(folder)
        save_path = os.path.join(folder, file_name)
        tts.save(save_path)
        return save_path
    except Exception as e:
        logging.error(f"Unexpected error: {e}")
        return None


def finalvoicefun(sceneList):
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
    dynamic_folder_voice = f"voice_{timestamp}"
    output_folder_voice = os.path.join("Generated_voice", dynamic_folder_voice)
    voice_paths = []
    for i, scene in enumerate(sceneList):
        try:
            voice_path = voice_generates(
                scene, f"voice_scene_{i+1}.mp3", output_folder_voice
            )

            if voice_path:
                voice_paths.append(voice_path)
                print(f"Voiceover saved locally: {voice_path}")
            else:
                print(f"Failed to save voiceover for scene {i+1}.")
        except Exception as e:
            print(f"Error generating voice for scene {i+1}: {e}")
    return output_folder_voice    

def generate_scence_voice(state:GraphState)->GraphState:
    sceneList = state['sceneList']
    resp = finalvoicefun(sceneList) 
    print(f"Generating voice{resp}")
    return state


workflow = StateGraph(GraphState)
workflow.add_node('generate_scence_voice',generate_scence_voice)
workflow.add_edge(START,'generate_scence_voice')
workflow.add_edge('generate_scence_voice',END)
app = workflow.compile()

sceneList = [
    "Mika, a boy with kind eyes, explores the lush jungle, playfully interacting with a troop of monkeys swinging through the trees.  A majestic banyan tree looms in the background.",
  ]  
resp = app.invoke({'sceneList':sceneList})
 

In [ ]:
import os
import requests
from datetime import datetime
import logging
from typing_extensions import TypedDict
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import StateGraph,START,END

class GraphState(TypedDict): 
    image_prompts: list[str]
    image_saved_path:str
    


def saveImage(image_content, save_path):
    output_folder = os.path.dirname(save_path)
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    with open(save_path, "wb") as f:
        f.write(image_content)


def images_generates(prompt: str):
    try:
        # response = client.images.generate(
        #     model=imagemodel,
        #     prompt=prompt,
        #     quality="standard",
        #     n=1,
        # )
        # image_url = response.data[0].url
        image_url = "https://www.w3schools.com/w3images/lights.jpg"
        return image_url

    except Exception as e:
        logging.error(f"Unexpected error generating image: {e}")
        raise


def finalscenesfun(image_prompts):
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
    dynamic_folder = f"images_{timestamp}"
    output_folder = os.path.join("Generated_images", dynamic_folder)

    for i, scene in enumerate(image_prompts):
        try:
             
            imageUrl = images_generates(scene)
            if not imageUrl:
                raise ValueError("Generated image URL is empty.")

            imageContent = requests.get(imageUrl).content
            if not imageContent:
                raise ValueError("Failed to retrieve image content.")

            save_path = os.path.join(output_folder, f"image_scene_{i+1}.png")
            saveImage(imageContent, save_path)
            print(f"Images saved to {save_path}")

        except Exception as e:
            logging.error(f"Error processing scene {i+1}: {e}")
            continue

    return output_folder

def generate_scence_image(state:GraphState)->GraphState:
    image_prompts = state['image_prompts']
    resp = finalscenesfun(image_prompts) 
    state["image_saved_path"]=resp
    print(f"Generating images path: {resp}")
    return state


workflow = StateGraph(GraphState)
workflow.add_node('generate_scence_image',generate_scence_image)
workflow.add_edge(START,'generate_scence_image')
workflow.add_edge('generate_scence_image',END)
app = workflow.compile()


image_prompts = [
    "Create a detailed, photorealistic image of the following scene: A young boy, Mika, with kind eyes and dark hair, playfully interacts with a troop of brown and grey monkeys swinging through the lush green canopy of a vibrant jungle.  A majestic, sprawling banyan tree with thick, gnarled roots dominates the background.  The scene is filled with sunlight dappling through the leaves.  Style: vibrant, whimsical illustration.",
    "Create a detailed, photorealistic image of the following scene: Under the shade of a massive banyan tree, a thin, weak lion cub, Leo, with dull eyes and ribs showing through his dusty fur, whimpers.  A young boy, Mika, kneels beside him, offering a juicy mango. The jungle floor is dappled with sunlight, and the air is humid. Style: realistic, slightly painterly.",
    "Create a detailed, photorealistic image of the following scene: Days later, a healthier lion cub, Leo, with thicker, shinier fur and bright eyes, playfully wrestles with Mika amidst the lush undergrowth of a tropical jungle.  Mika's father, a man with weathered skin and kind eyes, watches from a distance, a slight smile on his face. Style:  photorealistic, warm lighting.",
    "Create a detailed, photorealistic image of the following scene: A strong and vibrant lion cub, Leo, playfully tumbles with his magnificent lioness mother under the sprawling branches of a large banyan tree.  Mika, the kind boy who helped him, watches from a distance with a happy expression. The jungle is bathed in the golden light of late afternoon. Style:  impressionistic, focusing on light and color.",
    "Create a detailed, photorealistic image of the following scene: Mika, a boy with a gentle smile, observes from afar as Leo, now a healthy and playful lion cub with thick, glossy fur, frolics with his majestic lioness mother under a large banyan tree. The scene is peaceful and joyful, filled with the sounds of the jungle.  The banyan tree's roots spread wide, creating a natural haven. Style:  peaceful, serene illustration, emphasizing the bond between Leo and his mother.",
]
resp = app.invoke({'image_prompts':image_prompts}) 
print(resp)

In [55]:
import cv2
import numpy as np
import os
from moviepy import *
def zoom_in(image, num_frames=30, zoom_factor=0.5): 
    """Generates frames for a zoom-in effect."""
    frames = []
    h, w = image.shape[:2]
    for i in range(num_frames):
        scale = 1.0 + (i / num_frames) * zoom_factor
        center = (w // 2, h // 2)
        M = cv2.getRotationMatrix2D(center, 0, scale)
        frame = cv2.warpAffine(image, M, (w, h))
        frames.append(frame)
    return frames

def zoom_out(image, num_frames=30, zoom_factor=0.5): 
    """Generates frames for a zoom-out effect without blinking."""
    frames = []
    h, w = image.shape[:2]
    for i in range(num_frames):
        scale = 1.0 + ((num_frames - i - 1) / num_frames) * zoom_factor   
        center = (w // 2, h // 2)
        M = cv2.getRotationMatrix2D(center, 0, scale)
        frame = cv2.warpAffine(image, M, (w, h))
        frames.append(frame)
    return frames

def fade_in(image, num_frames=30): 
    """Creates a fade-in effect from black to the image."""
    frames = []
    black = np.zeros_like(image)
    for i in range(num_frames):
        alpha = i / num_frames
        frame = cv2.addWeighted(image, alpha, black, 1 - alpha, 0)
        frames.append(frame)
    return frames

def fade_out(image, num_frames=30): 
    """Creates a fade-out effect from image to black."""
    frames = []
    black = np.zeros_like(image)
    for i in range(num_frames):
        alpha = 1 - i / num_frames
        frame = cv2.addWeighted(image, alpha, black, 1 - alpha, 0)
        frames.append(frame)
    return frames
def load_voices(voices_root_folder):
    try:
        # Ensure paths are strings
        if not isinstance(voices_root_folder, str):
            raise ValueError("Voice folder path should be a string.") 
        
        voices = sorted([
            os.path.join(voices_root_folder, voice)
            for voice in os.listdir(voices_root_folder)
            if voice.lower().endswith('.mp3')
        ])
        return voices
    except Exception as e:
        raise ValueError(f"Error in load_voices function: {e}")
def images_to_video_with_effects(image_paths, output_video="output.mp4", fps=30, durations=None): 
    """Converts a list of images into a video with zoom and fade effects."""
    
    if not image_paths:
        raise ValueError("No images provided.") 
    
    first_img = cv2.imread(image_paths[0])
    if first_img is None:
        raise ValueError(f"Cannot load image: {image_paths[0]}")
        
    h, w, _ = first_img.shape 
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(output_video, fourcc, fps, (w, h))

    # Default durations if not provided
    if durations is None:
        durations = [2] * len(image_paths)  # Default 2 seconds per image

    for idx, (path, duration) in enumerate(zip(image_paths, durations)):
        img = cv2.imread(path)
        if img is None:
            print(f"Warning: Cannot load image {path}. Skipping.")
            continue 
        img = cv2.resize(img, (w, h))

        frame_duration = int(fps * duration)  # Convert duration to frames
        
        if idx == 0: 
            zoom_in_frames = zoom_in(img, num_frames=frame_duration, zoom_factor=0.5)  
            final_zoom_frame = zoom_in_frames[-1]  
            fade_out_frames = fade_out(final_zoom_frame, num_frames=int(fps * 1))  
            effect_frames = zoom_in_frames + fade_out_frames  
        else:  
            fade_in_frames = fade_in(img, num_frames=int(fps * 1))  
            fade_out_frames = fade_out(img, num_frames=int(fps * 1))  
            steady_frames = [img] * frame_duration  
            effect_frames = fade_in_frames + steady_frames + fade_out_frames
        print(f"Total frames written: {len(effect_frames)}")

        for frame in effect_frames:
            video_writer.write(frame)
    
    video_writer.release()
    print(f"Video saved as {output_video}")

# --- CONFIGURATION ---
image_folder = "Generated_images/images_20250320130205"
voice_generates_folder = "Generated_voice/voice_20250320130830"
output_path = "./final_outputs/output1254.mp4"
fps = 30 
voices = load_voices(voice_generates_folder)
image_paths = sorted([
    os.path.join(image_folder, img) 
    for img in os.listdir(image_folder) 
    if img.lower().endswith(('.png', '.jpg', '.jpeg'))
])

display_durations = []
for file_path in voices:
    try:
        audio = AudioFileClip(file_path)
        duration_sec = audio.duration
        display_durations.append(duration_sec)
        print(f"Duration of {file_path}: {duration_sec} seconds")
    except Exception as e:
        print(f"Error processing {file_path}: {e}")

# Generate video
images_to_video_with_effects(image_paths, output_video=output_path, fps=fps, durations=display_durations)


Duration of Generated_voice/voice_20250320130830\voice_scene_1.mp3: 13.75 seconds
Duration of Generated_voice/voice_20250320130830\voice_scene_2.mp3: 12.26 seconds
Duration of Generated_voice/voice_20250320130830\voice_scene_3.mp3: 11.35 seconds
Duration of Generated_voice/voice_20250320130830\voice_scene_4.mp3: 12.96 seconds
Duration of Generated_voice/voice_20250320130830\voice_scene_5.mp3: 13.92 seconds
Duration of Generated_voice/voice_20250320130830\voice_scene_6.mp3: 13.92 seconds
Total frames written: 442
Total frames written: 427
Total frames written: 400
Total frames written: 448
Total frames written: 477
Total frames written: 477
Video saved as ./final_outputs/output1254.mp4
